## LLM para Classificação de Documentos

In [ ]:
import os
API_KEY = 'XXX'
PATH_DATASET = 'XXXX'
MODELO_GPT = 'gpt-4o-mini'

os.environ['OPENAI_API_KEY'] = API_KEY

In [ ]:
import pandas as pd

df_teste = pd.read_csv(PATH_DATASET)
df_teste = df_teste[df_teste.subset == 'test']
df_teste

In [ ]:
from openai import OpenAI
import tqdm
import pandas as pd

prompts = { 'especialista' : 
                {'Despacho ou decisão saneador(a)' : 'Analise o texto fornecido e determine se ele corresponde a um despacho saneador ou a outro tipo de documento jurídico. O despacho saneador é um documento jurídico no qual o juiz constata a existência de elementos mínimos para que a fase probatória do processo possa ser iniciada, relatando o que já foi apresentado, corrigindo eventuais defeitos e indicando os próximos passos. Os elementos mínimos para o prosseguimento do processo verificados no despacho saneador são: se os atos até então praticados no processo são válidos e se há alguma questão processual pendente. Em seguida, o despacho saneador definirá quais são as questões jurídicas relevante para o julgamento; se há fatos incontroversos, que não mais precisarão ser discutidos, e quais são os fatos controversos, que precisarão, iniciada a fase probatória, ser provados pelas partes; quais serão as provas admitidas, se periciais e/ou testemunhais, e que deverão ser produzidas na fase de instrução; sobre quem recairá o dever de provar cada um dos fatos controversos; e, ao final, convocará um perito e/ou designará uma audiência de instrução. No despacho saneador, o juiz também poderá anunciar que julgará o processo antecipadamente, se chegar à conclusão de que as alegações das partes já estão provadas do modo como estão e são suficientes para uma decisão de mérito. Baseie sua resposta na estrutura, no conteúdo e no propósito do documento analisado, conforme as instruções.',
                 'Decisão Inicial': 'Analise o texto fornecido e determine se ele corresponde a uma decisão inicial ou a outro tipo de documento jurídico. Considere que a decisão inicial do processo é o documento jurídico emitido pelo juiz ou tribunal que, pela primeira vez, aceita ou nega, defere ou indefere um dos pedidos feitos pelo autor da ação. Essas decisões costumam ter como conteúdo o deferimento ou indeferimento de pedidos de tutela provisória de urgência ou de evidência. Em geral, as decisões iniciais trazem a seguinte expressão: “DECISÃO INTERLOCUTÓRIA”, que serve de título identificador do documento. Sempre que houver essa expressão e o documento resolver questões preliminares ou provisórias, como as “tutelas provisórias de urgência ou de evidência”, trata-se de uma decisão inicial. Baseie sua resposta na estrutura, no conteúdo e no propósito do documento analisado, conforme as instruções acima.',
                 'Réplica': 'Analise o texto fornecido e determine se ele corresponde a uma réplica ou a outro tipo de documento jurídico. Considere que réplica é a segunda manifestação do autor da ação, aquele que entrou com a petição inicial, e tem como objetivo responder aos argumentos e provas trazidos pelo réu na sua contestação. A réplica é um documento jurídico que busca refutar as alegações de uma contestação. Em geral, as réplicas trazem a seguinte expressão: “vem oferecer RÉPLICA À CONTESTAÇÃO”, que serve como título identificador da peça. Sempre que houver essa expressão, trata-se de um documento jurídico do tipo réplica. Baseie sua resposta na estrutura, no conteúdo e no propósito do documento analisado, conforme as instruções acima.',
                 'Petição Inicial': 'Analise o texto fornecido e determine se ele corresponde a uma petição inicial ou a outro tipo de documento jurídico. Considere que petição inicial é o primeiro documento jurídico do processo, pois é ele que inicia o processo. O propósito das petições iniciais é iniciar uma ação, oferecendo-a a um juiz ou tribunal. Por isso, é comum que as petições iniciais tragam a seguinte expressão: “vem oferecer AÇÃO”. Além disso, a petição inicial tem outros elementos que a distinguem de outros documentos jurídicos, como a menção a um valor da causa em reais e o pedido de citação do réu. Baseie sua resposta na estrutura, no conteúdo e no propósito do documento analisado, conforme as instruções acima.'
                 },
                
            'promptbreeder' : 
                {'Despacho ou decisão saneador(a)' : 'Analise o texto jurídico e determine se ele é um despacho saneador, destacando características essenciais como a inversão do ônus da prova e a definição das provas. Enriqueça sua interpretação com exemplos práticos e associações inusitadas, garantindo que a análise seja clara e acessível. Evite erros de classificação e finalize com uma conclusão que sintetize os principais pontos, enfatizando a importância da precisão na análise.',
                 'Decisão Inicial': 'Analise o texto de um documento jurídico e verifique se é uma decisão inicial, focando na expressão "DECISÃO INTERLOCUTÓRIA" e se trata de questões como tutelas de urgência. Considere também o impacto dessa decisão no processo judicial e nas partes envolvidas, mas de forma simplificada.',
                 'Réplica':  'Analise o texto de um documento jurídico apresentado e determine se ele se classifica como uma réplica à contestação. Para isso, verifique a presença de expressões como "vem oferecer RÉPLICA À CONTESTAÇÃO" e outros indicadores que evidenciem a intenção de refutar as alegações do réu. Avalie se o documento responde diretamente aos argumentos apresentados na contestação. Classifique o documento como réplica ou não, considerando seu contexto legal.',
                 'Petição Inicial': 'Analise o texto jurídico fornecido e determine se ele é uma petição inicial. Para isso, verifique se o documento contém a expressão “vem oferecer AÇÃO”, menciona um valor da causa em reais e solicita a citação do réu. Avalie a estrutura, o conteúdo e a intenção do documento, considerando que a petição inicial é o primeiro documento que inicia um processo judicial. Se a classificação não for correta, explique os erros encontrados e justifique sua resposta com base nas características típicas de uma petição inicial.'
                 } 
            }

client = OpenAI()

true_label = []
pred_label = []
result = []

tipos_documentos_avaliados = ['Despacho ou decisão saneador(a)', 'Decisão Inicial', 'Réplica', 'Petição Inicial']
experimentos = ['especialista', 'promptbreeder']

resultados = {}

for experimento in experimentos:
    for tipo_documento in tqdm.tqdm(tipos_documentos_avaliados):

        for id, row in tqdm.tqdm(df_teste.iterrows()):

            texto = row.texto
            
            prompt = prompts[experimento]
            prompt = prompt[tipo_documento]
            
            content = f"""
                        Analise a instrução a seguir e identifique se o ```texto``` pertence ao tipo de documento jurídico ```{tipo_documento}``` ou ```Não```.
                        {prompt}
                        
                        Respoda "Sim" se o texto representa um documento do tipo ```{tipo_documento}```, ou ```Não``` caso contrário.
                        
                        Texto:
                        {texto}
                        """
                            
            
            completion = client.chat.completions.create(
                            model=MODELO_GPT,
                            messages=[
                                {"role": "user", "content": content}
                                    ]
                        )

            output = completion.choices[0].message.content
            true = "Sim" if row.label == tipo_documento else "Não"
            
            result.append({'tipo_documento' : tipo_documento,
                            'experimento': experimento,
                            'label_true' : true,
                            'label_pred': output,
                            'texto': row.texto})

df_result = pd.DataFrame(result)
df_result

In [ ]:
from sklearn.metrics import classification_report

for experimento in experimentos:
    print(experimento.upper())
    for tipo_documento in tipos_documentos_avaliados:
        df_filtrado = df_result[(df_result.experimento == experimento) & (df_result.tipo_documento == tipo_documento)]
        print(tipo_documento)
        print(classification_report(df_filtrado.label_true, df_filtrado.label_pred))